# Week 14 (Monday), AST 8581 / PHYS 8581 / CSCI 8581 / STAT 8581: Big Data in Astrophysics

### Michael Coughlin <cough052@umn.edu>, Jie Ding <dingj@umn.edu>

With contributions derived from docs.python-guide.org and Cyrille Rossant (the IPython Interactive Computing and Visualization Cookbook)


# Where do we stand?

Foundations of Data and Probability -> Statistical frameworks (Frequentist vs Bayesian) -> Estimating underlying distributions -> Analysis of Time series (periodicity) -> Analysis of Time series (variability) -> Analysis of Time series (stochastic processes) -> Gaussian Processes -> Decision Trees / Regression -> Dimensionality Reduction -> Principle Component Analysis -> Clustering -> Density Estimation / Anomaly Detection -> Supervised Learning -> Deep Learning -> Introduction to Databases - SQL -> Introduction to Databases - NoSQL -> Introduction to Multiprocessing -> Introduction to GPUs -> Unit Testing

# Unit Testing

## Introduction

Untested code is broken code. Manual testing is essential to ensuring that our software works as expected and does not contain critical bugs. However, manual testing is severely limited because bugs may be introduced at any time in the code.

Nowadays, automated testing is a standard practice in software engineering. In this lesson, we will briefly cover important aspects of automated testing: unit tests, test-driven development, test coverage, and continuous integration. Following these practices is fundamental in order to produce high-quality software.

Python has a native unit testing module that you can readily use (unittest). Other third-party unit testing packages exist. In this recipe, we will use py.test. It is installed by default in Anaconda, but you can also install it manually with `conda install pytest`.

## How it works
By definition, a unit test must focus on one specific functionality. All unit tests should be completely independent. Writing a program as a collection of well-tested, mostly decoupled units forces you to write modular code that is more easily maintainable.

In a Python package, a test_xxx.py module should accompany every Python module named xxx.py. This testing module contains unit tests that test functionality implemented in the xxx.py module.

For a working example of this, compare the function suite:
https://github.com/skyportal/skyportal/tree/master/skyportal/handlers/api
with the corresponding set of test functions:
https://github.com/skyportal/skyportal/tree/master/skyportal/tests/api



Sometimes, your module's functions require preliminary work to run (for example, setting up the environment, creating data files, or setting up a web server). The unit testing framework can handle this via fixtures. The state of the system environment should be exactly the same before and after a testing module runs. If your tests affect the file system, they should do so in a temporary directory that is automatically deleted at the end of the tests. Testing frameworks such as py.test provide convenient facilities for this use-case.

For a working example of this, see e.g.
https://github.com/skyportal/skyportal/blob/master/skyportal/tests/conftest.py


Tests typically involve many assertions. With py.test, you can simply use the builtin assert keyword. Further convenient assertion functions are provided by NumPy (see http://docs.scipy.org/doc/numpy/reference/routines.testing.html). They are especially useful when working with arrays. For example, np.testing.assert_allclose(x, y) asserts that the x and y arrays are almost equal, up to a given precision that can be specified.

## Why Test?

Writing a full testing suite takes time. It imposes strong (but good) constraints on your code's architecture. It is a real investment, but it is always profitable in the long run. Also, knowing that your project is backed by a full testing suite is a real load off your mind.

* First, thinking about unit tests from the beginning forces you to think about a modular architecture. It is really difficult to write unit tests for a monolithic program full of interdependencies.

* Second, unit tests make it easier for you to find and fix bugs. If a unit test fails after introducing a change in the program, isolating and reproducing the bugs becomes trivial.

* Third, unit tests help you avoid regressions, that is, fixed bugs that silently reappear in a later version. When you discover a new bug, you should write a specific failing unit test for it. To fix it, make this test pass. Now, if the bug reappears later, this unit test will fail and you will immediately be able to address it.

When you write a complex program based on interdependent APIs, having a good test coverage for one module means that you can safely rely on it in other modules, without worrying about its behavior not conforming to its specification.

Unit tests are just one type of automated tests. Other important types of tests include integration tests (making sure that different parts of the program work together) and functional tests (testing typical use-cases).

## General rules of testing

* A testing unit should focus on one tiny bit of functionality and prove it correct.
* Each test unit must be fully independent. Each test must be able to run alone, and also within the test suite, regardless of the order that they are called. The implication of this rule is that each test must be loaded with a fresh dataset and may have to do some cleanup afterwards. This is usually handled by setUp() and tearDown() methods.
* Try hard to make tests that run fast. If one single test needs more than a few milliseconds to run, development will be slowed down or the tests will not be run as often as is desirable. In some cases, tests can’t be fast because they need a complex data structure to work on, and this data structure must be loaded every time the test runs. Keep these heavier tests in a separate test suite that is run by some scheduled task, and run all other tests as often as needed.
* Learn your tools and learn how to run a single test or a test case. Then, when developing a function inside a module, run this function’s tests frequently, ideally automatically when you save the code.
* Always run the full test suite before a coding session, and run it again after. This will give you more confidence that you did not break anything in the rest of the code.
* It is a good idea to implement a hook that runs all tests before pushing code to a shared repository.
* If you are in the middle of a development session and have to interrupt your work, it is a good idea to write a broken unit test about what you want to develop next. When coming back to work, you will have a pointer to where you were and get back on track faster.
* The first step when you are debugging your code is to write a new test pinpointing the bug. While it is not always possible to do, those bug catching tests are among the most valuable pieces of code in your project.
* Use long and descriptive names for testing functions. The style guide here is slightly different than that of running code, where short names are often preferred. The reason is testing functions are never called explicitly. square() or even sqr() is ok in running code, but in testing code you would have names such as test_square_of_number_2(), test_square_negative_number(). These function names are displayed when a test fails, and should be as descriptive as possible.
* When something goes wrong or has to be changed, and if your code has a good set of tests, you or other maintainers will rely largely on the testing suite to fix the problem or modify a given behavior. Therefore the testing code will be read as much as or even more than the running code. A unit test whose purpose is unclear is not very helpful in this case.
* Another use of the testing code is as an introduction to new developers. When someone will have to work on the code base, running and reading the related testing code is often the best thing that they can do to start. They will or should discover the hot spots, where most difficulties arise, and the corner cases. If they have to add some functionality, the first step should be to add a test to ensure that the new functionality is not already a working path that has not been plugged into the interface.

## How to do it

1.  Let's write in a first.py file a simple function that returns the first element of a list.

In [2]:
%%writefile first.py
def first(l):
    return l[0]

Overwriting first.py


2.  To test this function, we write another function, the unit test, that checks our first function using an example and an assertion:


In [6]:
%%writefile -a first.py

# This is appended to the file.
def test_first():
    assert first([1, 2, 3]) == 1

Appending to first.py


In [7]:
%cat first.py

def first(l):
    return l[0]
Overwriting first.py
%%writefile -a first.py


# This is appended to the file.
def test_first():
    assert first([1, 2, 3]) == 1


3.  To run the unit test, we use the pytest executable (the ! means that we're calling an external program from Jupyter):

In [10]:
!pytest first.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 1 item                                                               

first.py .                                                               [100%]

============================== 1 passed in 0.01s ===============================


4.  Our test passes! Let's add another example with an empty list. We want our function to return None in this case:

In [12]:
%%writefile first.py
def first(l):
    return l[0]

def test_first():
    assert first([1, 2, 3]) == 1
    assert first([]) is None

Overwriting first.py


In [13]:
!pytest first.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 1 item                                                               

first.py F                                                               [100%]

=================================== FAILURES ===================================
__________________________________ test_first __________________________________

    def test_first():
        assert first([1, 2, 3]) == 1
>       assert first([]) is None

first.py:6: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

l = []

    def first(l):
>       return l[0]
E       IndexError: list index out of range

first.py:2: IndexError
=========================== short test summary info ============================
FAILED fir

5.  This time, our test fails. Let's fix it by modifying the first() function:

In [14]:
%%writefile first.py
def first(l):
    return l[0] if l else None

def test_first():
    assert first([1, 2, 3]) == 1
    assert first([]) is None

Overwriting first.py


In [15]:
!pytest first.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 1 item                                                               

first.py .                                                               [100%]

============================== 1 passed in 0.01s ===============================


The test passes again!

## Pytest Fixtures

Fixtures are used when we want to run some code before every test method. So instead of repeating the same code in every test we define fixtures. Usually, fixtures are used to initialize database connections, pass the base , etc

A method is marked as a Pytest fixture by marking with

`@pytest.fixture`

A test method can use a Pytest fixture by mentioning the fixture as an input parameter.

Create a new file test_basic_fixture.py with following code

In [22]:
%%writefile -a test_fixtures.py

import pytest
@pytest.fixture
def supply_AA_BB_CC():
    aa=25
    bb =35
    cc=45
    return [aa,bb,cc]

def test_comparewithAA(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[0]==zz,"aa and zz comparison failed"

def test_comparewithBB(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[1]==zz,"bb and zz comparison failed"

def test_comparewithCC(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[2]==zz,"cc and zz comparison failed"

Writing test_fixtures.py


Here, we have a fixture named supply_AA_BB_CC. This method will return a list of 3 values.
We have 3 test methods comparing against each of the values.
Each of the test function has an input argument whose name is matching with an available fixture. Pytest then invokes the corresponding fixture method and the returned values will be stored in the input argument , here the list [25,35,45]. Now the list items are being used in test methods for the comparison.

Now run the test and see the result:

In [25]:
!py.test test_fixtures.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 3 items                                                              

test_fixtures.py F.F                                                     [100%]

=================================== FAILURES ===================================
______________________________ test_comparewithAA ______________________________

supply_AA_BB_CC = [25, 35, 45]

    def test_comparewithAA(supply_AA_BB_CC):
        zz=35
>       assert supply_AA_BB_CC[0]==zz,"aa and zz comparison failed"
E       AssertionError: aa and zz comparison failed
E       assert 25 == 35

test_fixtures.py:12: AssertionError
______________________________ test_comparewithCC ______________________________

supply_AA_BB_CC = [25, 35, 45]

    def

The test test_comparewithBB is passed since zz=BB=35, and the remaining 2 tests are failed.

The fixture method has a scope only within that test file it is defined. If we try to access the fixture in some other test file , we will get an error saying fixture ‘supply_AA_BB_CC’ not found for the test methods in other files.

To use the same fixture against multiple test files, we will create fixture methods in a file called conftest.py.

Let’s see this by the below PyTest example. Create 3 files conftest.py, test_basic_fixture.py, test_basic_fixture2.py with the following code

In [26]:
%%writefile -a conftest.py

import pytest
@pytest.fixture
def supply_AA_BB_CC():
    aa=25
    bb =35
    cc=45
    return [aa,bb,cc]

Writing conftest.py


In [27]:
%%writefile -a test_basic_fixture.py

import pytest
def test_comparewithAA(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[0]==zz,"aa and zz comparison failed"

def test_comparewithBB(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[1]==zz,"bb and zz comparison failed"

def test_comparewithCC(supply_AA_BB_CC):
    zz=35
    assert supply_AA_BB_CC[2]==zz,"cc and zz comparison failed"

Writing test_basic_fixture.py


In [28]:
%%writefile -a test_basic_fixture2.py

import pytest
def test_comparewithAA_file2(supply_AA_BB_CC):
	zz=25
	assert supply_AA_BB_CC[0]==zz,"aa and zz comparison failed"

def test_comparewithBB_file2(supply_AA_BB_CC):
	zz=25
	assert supply_AA_BB_CC[1]==zz,"bb and zz comparison failed"

def test_comparewithCC_file2(supply_AA_BB_CC):
	zz=25
	assert supply_AA_BB_CC[2]==zz,"cc and zz comparison failed"

Writing test_basic_fixture2.py


pytest will look for the fixture in the test file first and if not found it will look in the conftest.py

Run the test by py.test -k test_comparewith -v:

In [29]:
!py.test -k test_comparewith -v

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/mcoughlin/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 9 items                                                              

test_basic_fixture.py::test_comparewithAA FAILED                         [ 11%]
test_basic_fixture.py::test_comparewithBB PASSED                         [ 22%]
test_basic_fixture.py::test_comparewithCC FAILED                         [ 33%]
test_basic_fixture2.py::test_comparewithAA_file2 PASSED                  [ 44%]
test_basic_fixture2.py::test_comparewithBB_file2 FAILED                  [ 55%]
test_basic_fixture2.py::test_comparewithCC_file2 FAILED                  [ 66%]
test_fixtures.py::test_comparewithAA FAILED                              [ 77%]
test_fix

## Parameterized Tests


The purpose of parameterizing a test is to run a test against multiple sets of arguments. We can do this by @pytest.mark.parametrize.

We will see this with the below PyTest example. Here we will pass 3 arguments to a test method. This test method will add the first 2 arguments and compare it with the 3rd argument.

Create the test file test_addition.py with the below code

In [33]:
%%writefile -a test_addition.py

import pytest
@pytest.mark.parametrize("input1, input2, output",[(5,5,10),(3,5,12)])
def test_add(input1, input2, output):
    assert input1+input2 == output,"failed"

Writing test_addition.py


Here the test method accepts 3 arguments- input1, input2, output. It adds input1 and input2 and compares against the output.

Let’s run the test by py.test -k test_add -v and see the result

In [34]:
!py.test -k test_add -v

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/mcoughlin/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 11 items / 9 deselected / 2 selected                                 

test_addition.py::test_add[5-5-10] PASSED                                [ 50%]
test_addition.py::test_add[3-5-12] FAILED                                [100%]

=================================== FAILURES ===================================
_______________________________ test_add[3-5-12] _______________________________

input1 = 3, input2 = 5, output = 12

    @pytest.mark.parametrize("input1, input2, output",[(5,5,10),(3,5,12)])
    def test_add(input1, input2, output):
>       assert input1+input2 == output,"failed"
E       AssertionError: failed
E       ass

You can see the tests ran 2 times – one checking 5+5 ==10 and other checking 3+5 ==12

test_addition.py::test_add[5-5-10] PASSED

test_addition.py::test_add[3-5-12] FAILED

## xfail / Skip Tests

There will be some situations where we don’t want to execute a test, or a test case is not relevant for a particular time. In those situations, we have the option to Xfail the test or skip the tests

The xfailed test will be executed, but it will not be counted as part failed or passed tests. There will be no traceback displayed if that test fails. We can xfail tests using

@pytest.mark.xfail.

Skipping a test means that the test will not be executed. We can skip tests using

@pytest.mark.skip.

Try the test_addition_2.py with the below code

In [37]:
%%writefile -a test_addition_2.py

import pytest
@pytest.mark.skip
def test_add_1():
    assert 100+200 == 400,"failed"

@pytest.mark.skip
def test_add_2():
    assert 100+200 == 300,"failed"

@pytest.mark.xfail
def test_add_3():
    assert 15+13 == 28,"failed"

@pytest.mark.xfail
def test_add_4():
    assert 15+13 == 100,"failed"

def test_add_5():
    assert 3+2 == 5,"failed"

def test_add_6():
    assert 3+2 == 6,"failed"

Writing test_addition_2.py


Here

* test_add_1 and test_add_2 are skipped and will not be executed.
* test_add_3 and test_add_4 are xfailed. These tests will be executed and will be part of xfailed(on test failure) or xpassed(on test pass) tests. There won’t be any traceback for failures.
* test_add_5 and test_add_6 will be executed and test_add_6 will report failure with traceback while the test_add_5 passes

Execute the test by py.test test_addition_2.py -v and see the result

In [38]:
!py.test test_addition_2.py -v

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/mcoughlin/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/mcoughlin/Code/Teaching/AST8581/ast8581_2022_Spring_workspace/lecture/25
plugins: anyio-2.2.0, ligo.skymap-0.6.1
collected 6 items                                                              

test_addition_2.py::test_add_1 SKIPPED (unconditional skip)              [ 16%]
test_addition_2.py::test_add_2 SKIPPED (unconditional skip)              [ 33%]
test_addition_2.py::test_add_3 XPASS                                     [ 50%]
test_addition_2.py::test_add_4 XFAIL                                     [ 66%]
test_addition_2.py::test_add_5 PASSED                                    [ 83%]
test_addition_2.py::test_add_6 FAILED                                    [100%]

=================================== FAILURES ===================================
______

## Unit testing and continuous integration

A good habit to get into is running the full testing suite of our project at every commit. In fact, it is even possible to do this completely transparently and automatically through continuous integration. We can set up a server that automatically runs our testing suite in the cloud at every commit. If a test fails, we get an automatic e-mail telling us what the problem is so that we can fix it.

There are many continuous integration systems and services: Jenkins/Hudson, Travis CI (https://travis-ci.org), Codeship (http://codeship.com/), and others. Some of them play well with GitHub. For example, to use Github Actions with a GitHub project, one just needs to add appropriately formatted yml files in .github/workflows with various settings in your repository (see https://docs.github.com/en/actions/learn-github-actions/understanding-github-actions for details).

## Conclusion

In conclusion, unit testing, code coverage, and continuous integration are standard practices that should be used in all significant projects.

## In-class exercise: Write one or more unit tests for your group project
